In [1]:
# import required packages
# from fastai2.tabular.all import *
import xgboost as xgb
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.feature_extraction import DictVectorizer
from helpers import *

pd.options.display.max_columns = None

In [2]:
# path to project directory
path = Path('./')

In [3]:
# read in training dataset
train_df = pd.read_csv(path/'data/train_v7.csv', index_col=0, dtype={'season':str,
                                                                     'squad':str,
                                                                     'comp':str})

In [4]:
train_df.shape

(103431, 58)

In [5]:
# features for model
season = '2021'
cat_vars = ['season', 'position', 'was_home']
cont_vars = ['gw', 'minutes']
dep_var = ['total_points']

# find the latest gameweek
last_gw = train_df['gw'][train_df['season'] == season].max()

if np.isnan(last_gw): 
    next_gw = 1 
else: 
    next_gw = last_gw + 1
    
next_gw

23

In [6]:
# read in remaining_season.csv and sort
remaining_season_df = pd.read_csv(path/'data/remaining_season.csv', 
                                  index_col=0).sort_values(by=['gw', 'team'])

# update fields needed for processing
remaining_season_df['season'] = season
remaining_season_df['total_points'] = 0

# need to add an ordered date-like field  
remaining_season_df['kickoff_time'] = [str(9000 + x) + '-99-99' for x in remaining_season_df['match_no']]

# remove match_no, don't need it anymore
remaining_season_df = remaining_season_df.drop('match_no', axis=1)

# # append remaining season to end of training set
train_df = train_df.append(remaining_season_df, sort=False).reset_index(drop=True)

In [7]:
# # read in remaining_season.csv and sort
# remaining_season_df = pd.read_csv(path/'data/remaining_season.csv', 
#                                   index_col=0).sort_values(by=['gw', 'team'])

# remaining_season_df['season'] = season
# remaining_season_df['total_points'] = 0
# remaining_season_df['kickoff_time'] = '9999-99-99'

# # append remaining season to end of training set
# train_df = train_df.append(remaining_season_df, sort=False).reset_index(drop=True)

In [8]:
# add player lag features
lag_train_df, team_lag_vars = team_lag_features(train_df, ['total_points'], ['all', 3, 5, 10, 20])
lag_train_df, player_lag_vars = player_lag_features(lag_train_df, ['total_points'], ['all', 1, 2, 3, 5, 10, 20])

# add team conceded fields for opposition
pattern = re.compile('total_points_team_pg_last_.*_opponent')
team_lag_vars += [x.replace('team', 'team_conceded') for x in team_lag_vars if pattern.match(x)]

# but remove scored for opposition
pattern = re.compile('total_points_team_pg_last_.*_opponent')
team_lag_vars = [x for x in team_lag_vars if not pattern.match(x)]

In [9]:
team_lag_vars

['total_points_team_pg_last_all',
 'total_points_team_pg_last_3',
 'total_points_team_pg_last_5',
 'total_points_team_pg_last_10',
 'total_points_team_pg_last_20',
 'total_points_team_conceded_pg_last_all_opponent',
 'total_points_team_conceded_pg_last_3_opponent',
 'total_points_team_conceded_pg_last_5_opponent',
 'total_points_team_conceded_pg_last_10_opponent',
 'total_points_team_conceded_pg_last_20_opponent']

In [10]:
# we want to set gw and season as ordered categorical variables
# need lists with ordered categories
ordered_gws = list(range(1,39))
ordered_seasons = ['1617', '1718', '1819', '1920', '2021']

# set as categories with correct order 
lag_train_df['gw'] = lag_train_df['gw'].astype('category')
lag_train_df['season'] = lag_train_df['season'].astype('category')

lag_train_df['gw'].cat.set_categories(ordered_gws, ordered=True, inplace=True)
lag_train_df['season'].cat.set_categories(ordered_seasons, ordered=True, inplace=True)

In [11]:
lag_train_df[lag_train_df['player'] == 'Bernd Leno'].tail(25)

player  gw  position  minutes     team             opponent_team  \
97918   Bernd Leno  14         1     90.0  Arsenal                   Everton   
98530   Bernd Leno  15         1     90.0  Arsenal                   Chelsea   
99123   Bernd Leno  16         1     90.0  Arsenal  Brighton and Hove Albion   
99622   Bernd Leno  17         1     90.0  Arsenal      West Bromwich Albion   
100175  Bernd Leno  18         1     90.0  Arsenal            Crystal Palace   
100634  Bernd Leno  19         1     90.0  Arsenal          Newcastle United   
101553  Bernd Leno  20         1     90.0  Arsenal               Southampton   
102198  Bernd Leno  21         1     90.0  Arsenal         Manchester United   
102849  Bernd Leno  22         1     71.0  Arsenal   Wolverhampton Wanderers   
103438  Bernd Leno  23         1      0.0  Arsenal               Aston Villa   
104103  Bernd Leno  24         1      0.0  Arsenal                     Leeds   
104900  Bernd Leno  25         1      0.0  Arsenal           Manchester City   
105633  Bernd Leno  26         1      0.0  Arsenal            Leicester City   
106298  Bernd Leno  27         1      0.0  Arsenal                   Burnley   
106963  Bernd Leno  28         1      0.0  Arsenal         Tottenham Hotspur   
107691  Bernd Leno  29         1      0.0  Arsenal           West Ham United   
108356  Bernd Leno  30         1      0.0  Arsenal                 Liverpool   
109021  Bernd Leno  31         1      0.0  Arsenal          Sheffield United   
109686  Bernd Leno  32         1      0.0  Arsenal                    Fulham   
110351  Bernd Leno  33         1      0.0  Arsenal                   Everton   
111016  Bernd Leno  34         1      0.0  Arsenal          Newcastle United   
111745  Bernd Leno  35         1      0.0  Arsenal      West Bromwich Albion   
112410  Bernd Leno  36         1      0.0  Arsenal                   Chelsea   
113075  Bernd Leno  37         1      0.0  Arsenal            Crystal Palace   
113740  Bernd Leno  38         1      0.0  Arsenal  Brighton and Hove Albion   

        relative_market_value_team  relative_market_value_opponent_team  \
97918                     1.317380                             1.145049   
98530                     1.317380                             1.859440   
99123                     1.316625                             0.541862   
99622                     1.261632                             0.258537   
100175                    1.264284                             0.475724   
100634                    1.272515                             0.604033   
101553                    1.343890                             0.538945   
102198                    1.353529                             1.679166   
102849                    1.253765                             0.969065   
103438                    1.253765                             0.749770   
104103                    1.253765                             0.440130   
104900                    1.253765                             2.398901   
105633                    1.253765                             1.060269   
106298                    1.253765                             0.343201   
106963                    1.253765                             1.628650   
107691                    1.253765                             0.614096   
108356                    1.253765                             2.534688   
109021                    1.253765                             0.387898   
109686                    1.253765                             0.477517   
110351                    1.253765                             1.112321   
111016                    1.253765                             0.630730   
111745                    1.253765                             0.299523   
112410                    1.253765                             1.818752   
113075                    1.253765                             0.504448   
113740                    1.253765                      

In [12]:
# create dataset with latest lag numbers for all future gameweeks
valid_season = '2021'
valid_gw = next_gw
valid_len = 38 - next_gw + 1

train_predict_df, train_idx, valid_idx = create_lag_train(lag_train_df, 
                                                      cat_vars, cont_vars, 
                                                      player_lag_vars, team_lag_vars, dep_var,
                                                      valid_season, valid_gw, valid_len)

In [13]:
team_lag_vars

['total_points_team_pg_last_all',
 'total_points_team_pg_last_3',
 'total_points_team_pg_last_5',
 'total_points_team_pg_last_10',
 'total_points_team_pg_last_20',
 'total_points_team_conceded_pg_last_all_opponent',
 'total_points_team_conceded_pg_last_3_opponent',
 'total_points_team_conceded_pg_last_5_opponent',
 'total_points_team_conceded_pg_last_10_opponent',
 'total_points_team_conceded_pg_last_20_opponent']

In [14]:
# take a look at the dataframe
train_predict_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 114398 entries, 0 to 114397
Data columns (total 33 columns):
 #   Column                                           Non-Null Count   Dtype   
---  ------                                           --------------   -----   
 0   gw                                               114398 non-null  category
 1   minutes                                          114398 non-null  float64 
 2   minutes_last_1                                   114398 non-null  float64 
 3   minutes_last_10                                  114398 non-null  float64 
 4   minutes_last_2                                   114398 non-null  float64 
 5   minutes_last_20                                  114398 non-null  float64 
 6   minutes_last_3                                   114398 non-null  float64 
 7   minutes_last_5                                   114398 non-null  float64 
 8   minutes_last_all                                 114398 non-null  float64 
 9   oppo

In [15]:
# split out dependent variable
X, y = train_predict_df[cat_vars + cont_vars + player_lag_vars + team_lag_vars].copy(), train_predict_df[dep_var].copy()

In [16]:
# since position is categorical, it should be a string
X['position'] = X['position'].apply(str)

# need to transform season
enc = LabelEncoder()
X['season'] = enc.fit_transform(X['season'])
X_dict = X.to_dict("records")

# Create the DictVectorizer object: dv
dv = DictVectorizer(sparse=False, separator='_')

# Apply dv on df: df_encoded
X_encoded = dv.fit_transform(X_dict)

X_df = pd.DataFrame(X_encoded, columns=dv.feature_names_)

In [17]:
# split out training and prediction sets
X_train = X_df.iloc[train_idx]
y_train = y.iloc[train_idx]
X_test = X_df.iloc[valid_idx]
y_test = y.iloc[valid_idx]

In [18]:
# instatiate and train XGB Regressor
model_params = {"gamma": 0.42,
                "learning_rate": 0.047, 
                "max_depth": 4,
                "n_estimators": 171,
                "subsample": 0.6}

xg_reg = xgb.XGBRegressor(objective="reg:squarederror",
                          gamma=model_params['gamma'], 
                          learning_rate=model_params['learning_rate'], 
                          max_depth=model_params['max_depth'],
                          n_estimators=model_params['n_estimators'],
                          subsample=model_params['subsample'])

xg_reg.fit(X_train, y_train)
preds = xg_reg.predict(X_test)

In [19]:
preds

array([ 0.00591558, -0.0064798 , -0.02694178, ...,  0.00599194,
        3.5030873 ,  0.00599194], dtype=float32)

In [20]:
remaining_season_df['predicted_points'] = preds

In [21]:
remaining_season_df.head(50)

gw         team opponent_team                            player  \
5640  23      Arsenal   Aston Villa                        Mesut Özil   
5641  23      Arsenal   Aston Villa         Sokratis Papastathopoulos   
5642  23      Arsenal   Aston Villa        David Luiz Moreira Marinho   
5643  23      Arsenal   Aston Villa         Pierre-Emerick Aubameyang   
5644  23      Arsenal   Aston Villa                     Cédric Soares   
5645  23      Arsenal   Aston Villa               Alexandre Lacazette   
5646  23      Arsenal   Aston Villa                  Shkodran Mustafi   
5647  23      Arsenal   Aston Villa                        Bernd Leno   
5648  23      Arsenal   Aston Villa                      Granit Xhaka   
5649  23      Arsenal   Aston Villa                        Pablo Marí   
5650  23      Arsenal   Aston Villa                   Héctor Bellerín   
5651  23      Arsenal   Aston Villa                    Calum Chambers   
5652  23      Arsenal   Aston Villa                    Sead Kolasinac   
5653  23      Arsenal   Aston Villa                       Rob Holding   
5654  23      Arsenal   Aston Villa                    Kieran Tierney   
5655  23      Arsenal   Aston Villa                      Nicolas Pépé   
5656  23      Arsenal   Aston Villa                    Lucas Torreira   
5657  23      Arsenal   Aston Villa                      Reiss Nelson   
5658  23      Arsenal   Aston Villa                    Edward Nketiah   
5659  23      Arsenal   Aston Villa                  Emile Smith Rowe   
5660  23      Arsenal   Aston Villa                       Bukayo Saka   
5661  23      Arsenal   Aston Villa                  Matteo Guendouzi   
5662  23      Arsenal   Aston Villa  Gabriel Teodoro Martinelli Silva   
5663  23      Arsenal   Aston Villa                    William Saliba   
5664  23      Arsenal   Aston Villa                       Mathew Ryan   
5665  23      Arsenal   Aston Villa           Willian Borges Da Silva   
5666  23      Arsenal   Aston Villa                 Gabriel Magalhães   
5667  23      Arsenal   Aston Villa         Daniel Ceballos Fernández   
5668  23      Arsenal   Aston Villa     Mohamed Naser El Sayed Elneny   
5669  23      Arsenal   Aston Villa                        Matt Macey   
5670  23      Arsenal   Aston Villa              Rúnar Alex Rúnarsson   
5671  23      Arsenal   Aston Villa                     Thomas Partey   
5672  23      Arsenal   Aston Villa                   Martin Ødegaard   
169   23  Aston Villa       Arsenal                 Emiliano Martínez   
170   23  Aston Villa       Arsenal                        Tom Heaton   
171   23  Aston Villa       Arsenal                 Ahmed El Mohamady   
172   23  Aston Villa       Arsenal                    Henri Lansbury   
173   23  Aston Villa       Arsenal                       Neil Taylor   
174   23  Aston Villa       Arsenal                         Jed Steer   
175   23  Aston Villa       Arsenal                   Conor Hourihane   
176   23  Aston Villa       Arsenal    José Ignacio Peleteiro Romallo   
177   23  Aston Villa       Arsenal                      Ørjan Nyland   
178   23  Aston Villa       Arsenal                      Bjorn Engels   
179   23  Aston Villa       Arsenal                     Jack Grealish   
180   23  Aston Villa       Arsenal                       John McGinn   
181   23  Aston Villa       Arsenal                     Kortney Hause   
182   23  Aston Villa       Arsenal      Mahmoud Ahmed Ibrahim Hassan   
183   23  Aston Villa       Arsenal                      Tyrone Mings   
184   23  Aston Villa       Arsenal                      Matt Targett   
185   23  Aston Villa       Arsenal                        James Bree   

      position  price  play_proba   short_name  was_home  \
5640         3    6.7         0.0         Özil     False   
5641         2    4.8         0.0     Sokratis     False   
5642         2    5.4         0.0   David Luiz     False   
5643         3   11.3         1.0   Aubame

In [22]:
remaining_season_df[remaining_season_df['gw'] == next_gw].sort_values(by='predicted_points', ascending=False).head(50)

gw                      team             opponent_team  \
280   23         Manchester United                   Everton   
80    23         Tottenham Hotspur      West Bromwich Albion   
5645  23                   Arsenal               Aston Villa   
5784  23           Manchester City                 Liverpool   
5643  23                   Arsenal               Aston Villa   
5723  23           West Ham United                    Fulham   
9     23                     Leeds            Crystal Palace   
5519  23            Leicester City   Wolverhampton Wanderers   
73    23         Tottenham Hotspur      West Bromwich Albion   
5578  23                   Chelsea          Sheffield United   
164   23          Newcastle United               Southampton   
5734  23           West Ham United                    Fulham   
94    23         Tottenham Hotspur      West Bromwich Albion   
281   23         Manchester United                   Everton   
317   23                 Liverpool           Manchester City   
306   23                 Liverpool           Manchester City   
10    23                     Leeds            Crystal Palace   
5779  23           Manchester City                 Liverpool   
209   23                   Burnley  Brighton and Hove Albion   
5611  23               Southampton          Newcastle United   
28    23                     Leeds            Crystal Palace   
5529  23            Leicester City   Wolverhampton Wanderers   
217   23                   Burnley  Brighton and Hove Albion   
5758  23                   Everton         Manchester United   
5717  23           West Ham United                    Fulham   
270   23         Manchester United                   Everton   
5760  23                   Everton         Manchester United   
179   23               Aston Villa                   Arsenal   
5789  23           Manchester City                 Liverpool   
51    23   Wolverhampton Wanderers            Leicester City   
5526  23            Leicester City   Wolverhampton Wanderers   
5721  23           West Ham United                    Fulham   
202   23                   Burnley  Brighton and Hove Albion   
5659  23                   Arsenal               Aston Villa   
5786  23           Manchester City                 Liverpool   
5687  23  Brighton and Hove Albion                   Burnley   
7     23                     Leeds            Crystal Palace   
312   23                 Liverpool           Manchester City   
5738  23           West Ham United                    Fulham   
197   23               Aston Villa                   Arsenal   
5740  23           West Ham United                    Fulham   
141   23          Newcastle United               Southampton   
291   23         Manchester United                   Everton   
233   23                    Fulham           West Ham United   
5739  23           West Ham United                    Fulham   
5601  23                   Chelsea          Sheffield United   
221   23                   Burnley  Brighton and Hove Albion   
284   23         Manchester United                   Everton   
5718  23           West Ham United                    Fulham   
169   23               Aston Villa                   Arsenal   

                                       player  position  price  play_proba  \
280             Bruno Miguel Borges Fernandes         3   11.4         1.0   
80                              Heung-Min Son         3    9.7         1.0   
5645                      Alexandre Lacazette         4    8.4         1.0   
5784                          Raheem Sterling         3   11.6         1.0   
5643                Pierre-Emerick Aubameyang         3   11.3         1.0   
5723                          Michail Antonio         4    6.7         1.0   
9                             Patrick Bamford         4    6.6         1.0   
5519                              Jamie Vardy         4   10.0         1.0   
73                                Hugo 

In [23]:
# rename previous week's predictions file
# generate previous week's filename
last_gw = next_gw - 1
filename = 'history/2020-21/predictions_gw' + str(last_gw) + '.csv'
# windows
! move predictions.csv $filename
# linux
# ! mv predictions.csv $filename

        1 file(s) moved.


In [24]:
# write to predictions.csv
## RENAME PREVIOUS WEEK FIRST
predictions = remaining_season_df[['gw', 'player', 'team', 'opponent_team', 
                                   'position', 'price', 'play_proba', 'short_name',
                                   'was_home', 'predicted_points', 'minutes']]
predictions.to_csv('predictions.csv')